# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [36]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [37]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [38]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.prestige, df.admit)

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [129]:
#First cast the prestige column to an integer to more easiy manipulate
#is there a better way to do this? getting a warning
df['prestige'] = df['prestige'].astype(int)
dummy_ranks = pd.get_dummies(df.prestige, prefix='prestige')
dummy_ranks.head()

C:\Users\aaron\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,prestige_1,prestige_2,prestige_3,prestige_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: The number of unique categories minus 1

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [55]:
#dummy_ranks.head()
dummy_ranks.ix[:, 'prestige_1':].head()

C:\Users\aaron\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


,prestige_1,prestige_2,prestige_3,prestige_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


In [56]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [61]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pr1_ct = pd.crosstab(handCalc.prestige_1, handCalc.admit)
pr1_ct

admit,0,1
prestige_1,,
0,243,93
1,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [91]:
num_admit_pr1 = pr1_ct.loc[1,1]
total_pr1 = pr1_ct.loc[1,:].sum()

print 'Admit percentage if prestige=1: ' + str(round(num_admit_pr1 * 1.0 / total_pr1 * 100,1)) + "%"

Admit percentage if prestige=1: 54.1%


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [90]:
num_admit_pr0 = pr1_ct.loc[0,1]
total_pr1 = pr1_ct.loc[0,:].sum()

print 'Admit percentage if prestige!=1: ' + str(round(num_admit_pr1 * 1.0 / total_pr1 * 100,1)) + "%"

Admit percentage if prestige!=1: 9.8%


#### 3.3 Calculate the odds ratio

In [95]:
# odds ratio in this case would be (admit_p1 / rejected_p1) / (admit_p!=1 / rejected_p!=1)
print 'Odds ratio: ' + str((pr1_ct.loc[1,1]*1.0/pr1_ct.loc[1,0]*1.0)/(pr1_ct.loc[0,1]*1.0/pr1_ct.loc[0,0]))

Odds ratio: 3.07949308756


#### 3.4 Write this finding in a sentenance: 

Answer: You are 3.1 times more likely to be admitted into college if you went to a prestige 1 school compared to not going to a prestige 1 school

#### 3.5 Print the cross tab for prestige_4

In [97]:
pr4_ct = pd.crosstab(handCalc.prestige_4, handCalc.admit)
pr4_ct

admit,0,1
prestige_4,,
0,216,114
1,55,12


#### 3.6 Calculate the OR 

In [98]:
print 'Odds ratio: ' + str((pr4_ct.loc[1,1]*1.0/pr4_ct.loc[1,0]*1.0)/(pr4_ct.loc[0,1]*1.0/pr4_ct.loc[0,0]))

Odds ratio: 0.413397129187


#### 3.7 Write this finding in a sentence

Answer: You are 0.4 times or 60% less likely to be admitted into college if you went to a school of prestige 4 (compared to schools of all other prestiges)

## Part 4. Analysis

In [99]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           1           0
1      1  660.0  3.67           0           1           0
2      1  800.0  4.00           0           0           0
3      1  640.0  3.19           0           0           1
4      0  520.0  2.93           0           0           1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [101]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4,intercept
0,0,380.0,3.61,0,1,0,1.0
1,1,660.0,3.67,0,1,0,1.0
2,1,800.0,4.00,0,0,0,1.0
3,1,640.0,3.19,0,0,1,1.0
4,0,520.0,2.93,0,0,1,1.0


#### 4.1 Set the covariates to a variable called train_cols

In [102]:
train_cols = ['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4']

#### 4.2 Fit the model

In [106]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()


Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


#### 4.3 Print the summary results

In [105]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Tue, 29 Aug 2017   Pseudo R-squ.:                 0.05722
Time:                        22:24:56   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0014      0.001      1.308      0.191      -0.001       0.003
gpa           -0.1323      0.195     -0.680      0.497      -0.514       0.249
prestige_2    -0.9562      0.302     -3.171      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [107]:
print np.exp(result.params)

gre           1.001368
gpa           0.876073
prestige_2    0.384342
prestige_3    0.214918
prestige_4    0.154135
dtype: float64


In [122]:
#thus the range of odds ratio within the 95% interval is
print np.exp(result.conf_int()[0])
print
print np.exp(result.conf_int()[1])

gre           0.999320
gpa           0.598303
prestige_2    0.212826
prestige_3    0.112055
prestige_4    0.070176
Name: 0, dtype: float64

gre           1.003420
gpa           1.282800
prestige_2    0.694082
prestige_3    0.412207
prestige_4    0.338540
Name: 1, dtype: float64


In [120]:
#also realized i could do it this way which is cleaner but yields the same results
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                2.5%     97.5%        OR
gre         0.999320  1.003420  1.001368
gpa         0.598303  1.282800  0.876073
prestige_2  0.212826  0.694082  0.384342
prestige_3  0.112055  0.412207  0.214918
prestige_4  0.070176  0.338540  0.154135


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds ratio for prestige_2 is 0.384, which means that there is a 62% less chance of getting into school compare to prestige_1

However, within the 95% interval the range varies a lot and can range from a 79%% less chance (0.21) to a 31% less  chance (0.69)

#### 4.6 Interpret the OR of GPA

Answer: For every additional point in your GRE, your odds of getting in improve by 1

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [123]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [143]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [144]:
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


#### 5.1 Recreate the dummy variables

In [145]:
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
combos['prestige'] = combos['prestige'].astype(int)
combos.head()

,gre,gpa,prestige,intercept
0,220.0,2.260000,1,1.0
1,220.0,2.260000,2,1.0
2,220.0,2.260000,3,1.0
3,220.0,2.260000,4,1.0
4,220.0,2.453333,1,1.0


In [146]:
# recreate the dummy variables

dummy_ranks = pd.get_dummies(combos.prestige, prefix='prestige')
dummy_ranks.head()
# keep only what we need for making predictions
cols_to_keep2 = ['gre', 'gpa']
combos_indvar = combos[cols_to_keep2].join(dummy_ranks.ix[:, 'prestige_2':])
#combos_indvar['intercept'] = 1.0
combos_indvar.head()

C:\Users\aaron\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  import sys


,gre,gpa,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,0,0,0
1,220.0,2.260000,1,0,0
2,220.0,2.260000,0,1,0
3,220.0,2.260000,0,0,1
4,220.0,2.453333,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [150]:
combos_indvar['admit_pred'] = result.predict(combos_indvar[train_cols])
combos_indvar.tail(4)

,gre,gpa,prestige_2,prestige_3,prestige_4,admit_pred
396,800.0,4.0,0,0,0,0.637399
397,800.0,4.0,1,0,0,0.403204
398,800.0,4.0,0,1,0,0.274202
399,800.0,4.0,0,0,1,0.213184


#### 5.3 Interpret findings for the last 4 observations

Answer: If you have a perfect GPA and GRE, the probability of admission goes from 64% for a prestige=1 school to 21% for a prestige=4 school

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [159]:
## having issues with the pivot table :(

def strat_plot(var):
    grouped = pd.pivot_table(combos_indvar, values=['admit_pred'], index=[var, 'prestige'], aggfunc=np.mean)
    
    colors = 'rbgyrbgy'
    for col in combos_indvar.prestige.unique():
        plt_data = grouped.ix[grouped.index.get_level_values(1)==col]
        pl.plot(plt_data.index.get_level_values(0), plt_data['admit_pred'],
                color=colors[int(col)])

strat_plot('gre')
strat_plot('gpa')

KeyError: 'prestige'